# Item Based Recommender System  
-------------
_Author: Carleton Smith_

## Project Guide  
-------------
 - [Project Overview](#project-overview)
 - [Reading in the Data](#read-in)
     - [EDA](#eda)  
       
       
 - [Functioning of Recommender Systems](#simple)
     - [Predicting Unknown Scores](#pred)  
     
     
 - [Return to the Data](#ret)
 - [Utilization Complexities](#util)
 - [SVD](#svd)
 - [Surprise - Python package](#surprise)  
 
 <a id = "project-overview"></a>
 ## Project Overview  
 --------  
 #### EXPECTED TIME: 2.5 HRS  
 
 This project consists of 4 parts:  
 
 - General Familiarization with the data.
 - Mathematical foundations of recommender systems and simple examples about them.
 - Execution of methods on the data.
 - Short introduction to the `Surprise` package.   
 
The methods used below should all be familiar from the lectures in week 9. Except for SVD and the `Surprise` package, we will review some main concepts before demonstration.

The general goal of this project is, given a collection of users, items, and user reviews of the items, predict what score a user would assign to an item they have yet to review.  

We will be working with a synthetic review dataset, modeled on reviews from Amazon. To demonstrate the techniques, in all our examples, we will work only with a smaller portion of the made-up data.

**Motivation:** Recommender systems provide a non-parametric comparisons between items. They are fundamentals in analyzing how individuals can be served.

**Objectives:**  

- Understand mathematical foundations of recommender systems.
- Translation of mathematical algorithm into code.

<a id="questions"></a>
## Questions

+ [Question 01](#q01)
+ [Question 02](#q02)
+ [Question 03](#q03)
+ [Question 04](#q04)
+ [Question 05](#q05)
+ [Question 06](#q06)
+ [Question 07](#q07)
+ [Question 08](#q08)
+ [Question 09](#q09)

### Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

### Reading the data
In the first part of this assignment, we will be using synthetic data modeled from Amazon reviews. A description of the review data and instructions to download the database [can be found here](https://s3.amazonaws.com/amazon-reviews-pds/readme.html).  

Below we display a sample of the data taken from Amazon.

In [2]:
sample_data_path = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv"

rev_df = pd.read_table(sample_data_path)

rev_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,18778586,RDIJS7QYB6XNR,B00EDBY7X8,122952789,Monopoly Junior Board Game,Toys,5,0,0,N,Y,Five Stars,Excellent!!!,2015-08-31
1,US,24769659,R36ED1U38IELG8,B00D7JFOPC,952062646,56 Pieces of Wooden Train Track Compatible wit...,Toys,5,0,0,N,Y,Good quality track at excellent price,Great quality wooden track (better than some o...,2015-08-31
2,US,44331596,R1UE3RPRGCOLD,B002LHA74O,818126353,Super Jumbo Playing Cards by S&S Worldwide,Toys,2,1,1,N,Y,Two Stars,Cards are not as big as pictured.,2015-08-31
3,US,23310293,R298788GS6I901,B00ARPLCGY,261944918,Barbie Doll and Fashions Barbie Gift Set,Toys,5,0,0,N,Y,my daughter loved it and i liked the price and...,my daughter loved it and i liked the price and...,2015-08-31
4,US,38745832,RNX4EXOBBPN5,B00UZOPOFW,717410439,Emazing Lights eLite Flow Glow Sticks - Spinni...,Toys,1,1,1,N,Y,DONT BUY THESE!,Do not buy these! They break very fast I spun ...,2015-08-31


#### Looking at our raw data

Although Natural Language Processing (NLP) might offer insight into the structure of reviews, for this assignment, we are only interested in the customers (`costumer_id`), the products (`product_id`), and the scores (`star_rating`) that each costumer assigned to a particular product.

In [3]:
rev_df = rev_df[['customer_id','product_id', 'star_rating']]
rev_df.rename(columns= {'star_rating':'score'}, inplace=True)
rev_df.head()

,customer_id,product_id,score
0,18778586,B00EDBY7X8,5
1,24769659,B00D7JFOPC,5
2,44331596,B002LHA74O,2
3,23310293,B00ARPLCGY,5
4,38745832,B00UZOPOFW,1


<a id="q01"></a>
[Return to top](#questions)

### Question 01

Above, in our `ref_df` Database, we used the attribute `rename` to rename the column `star_rating` as `score` by passing a dictionary as a parameter.

Which other object, when passed to `columns`, would have yielded the same result?

- a) "reviewerID, productID, score"
- b) "['asin':'productID','overall':'score']"
- c) "['customer_id', 'product_id', 'score']"
- d) None of the above.

Assign the letter associated with your choice as a string to ans1.

In [8]:
### GRADED
#Which other object, when passed to `columns`, would have yielded the same result?
#a) "reviewerID, productID, score"
#b) "['asin':'productID','overall':'score']"
#c) "['customer_id', 'product_id', 'score']"
#d) None of the above.

#Assign the letter associated with your choice as a string to ans1.
### YOUR SOLUTION HERE
ans1 = "b"
###
### YOUR CODE HERE
###


In [9]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


Although this is not strictly necessary, we can modify the values in columns customer_id and product_id to make them a little easier to read.  

We will do so by building two dictionaries: one for `costumer_id` and the other one for `product_id` in the following way.

Let the keys for your dictionary be all the *n unique* values of `costumer_id`. Rewrite such values to read "R" followed by a number ###, from 0 to n, such that each unique key is mapped to a unique string of the format `R###`. We do the same for the products, but using "P" as a prefix instead.

In [10]:
rID_dict = {rID:'R'+ str(idx) for idx, rID in enumerate(rev_df['customer_id'].unique()) }
pID_dict = {pID:'P'+ str(idx) for idx, pID in enumerate(rev_df['product_id'].unique()) }


### Checking the values in the dictionaries below for products.
for k in list(pID_dict.keys())[:5]:
    print(k, ":",pID_dict[k])
print("length: ",len(pID_dict))

B00EDBY7X8 : P0
B00D7JFOPC : P1
B002LHA74O : P2
B00ARPLCGY : P3
B00UZOPOFW : P4
length:  49


Next, we rename the costumers and products IDs in our dataset.

In [11]:
rev_df.loc[:, 'product_id'] = rev_df.loc[:,'product_id'].map(pID_dict)
rev_df.loc[:, 'customer_id'] = rev_df.loc[:,'customer_id'].map(rID_dict)

rev_df.head()

,customer_id,product_id,score
0,R0,P0,5
1,R1,P1,5
2,R2,P2,2
3,R3,P3,5
4,R4,P4,1


<a id = "Exploratory Data Analysis"></a>
### Exploratory Data Analysis (EDA)
Having gone through the process of cleaning and analyzing the data, we will load a more extensive synthetic dataset and perform some light EDA to look at the distributions of the data.

In [12]:
# Reading the data
synth_data_path = "./data/synthetic_reviews.csv"

rev_df = pd.read_csv(synth_data_path)
rev_df.head()

FileNotFoundError: [Errno 2] File b'./data/synthetic_reviews.csv' does not exist: b'./data/synthetic_reviews.csv'

First, we visualize how many products got a score of 5, 4, 3, 2, or 1. 

In [ ]:
rev_df['score'].value_counts().plot(kind = 'bar');

Next, we count how many products received a specific score.

In [ ]:
rev_df['score'].value_counts()

We notice that there is an interesting preponderance of products with score 5. 

We continue by ranking the product from the ones that had the most reviews to the ones that had the least. We display the first 15 and the last 5 rows of this ranking.

In [ ]:
print("Reviews per product, top and bottom reviewed\n")
#Display first 15 rows
print(rev_df['productID'].value_counts()[:15])
#Display last 5 rows
print(rev_df['productID'].value_counts()[-5:])

Below, we visualize the ranking defined above.

In [ ]:
product_w_n_reviews = rev_df['productID'].value_counts().value_counts()
plt.figure(figsize = (10,7))
plt.scatter(product_w_n_reviews.index, product_w_n_reviews)
plt.xlabel("Number of Reviews", fontsize = 16)
plt.ylabel("Number of Products", fontsize = 16);

The above graph seem to be on a log-normal scale. 

In reality, the graph displayed has the shape of a "hollow curve". The log-normal appearance is an artifact of the data synthesis.

We will continue our EDA by ranking which costumers gave the most and least number of reviews.

In [ ]:
print("Reviews per reviewer, top and bottom reviewers\n")
print(rev_df['reviewerID'].value_counts()[:15])
print(rev_df['reviewerID'].value_counts()[-5:])

Again, we visualize the ranking defined above.

In [ ]:
product_w_n_reviews = rev_df['reviewerID'].value_counts().value_counts()
plt.figure(figsize = (10,7))
plt.scatter(product_w_n_reviews.index, product_w_n_reviews)
plt.xlabel("Number of Reviews", fontsize = 16)
plt.ylabel("Number of Users", fontsize = 16);

As expected, most products are only reviewed a few times, most users only submit a few reviews, and very few users have submitted many reviews. 

Again. the above graph has the shape of a "hollow curve".

<a id = "simple"></a>
### Functioning of Recommender Systems

Before tackling our extensive synthetic data, we will use a simpler dataset to demonstrate the functioning of recommender systems.  

Below we have a set of rankings of six musicians made by six individuals.  

In [ ]:
example_path = "./data/example.csv"
ex = pd.read_csv(example_path, index_col = 0)
ex

Note that the value "0" in the rows above denotes the absence of ranking - NOT an extremely poor ranking.  

To determine which product to recommend, we can compute the similarity scores between products. Similarity scores can be computed in three different ways:

- the Euclidean distance,
- the Pearson's correlation coefficient,
- and the cosine similarity score. 

To compare these scores we will normalize each to range between 0 and 1.  
To simplify our analysis, we will use a subset of our example DataFrame:

In [ ]:
ex_hand = ex.iloc[:3,-3:]
ex_hand

#### Euclidean Distance

The Euclidean distance between and point $i =(a_i, b_i, \dots, n_i)$ and $j= (a_j, b_j, \dots, n_j)$ is given by

$$\sqrt{(a_j-a_i)^2+(b_j-b_i)^2+...+(n_j-n_i)^2}$$  


<a id="q02"></a>
[Return to top](#questions)
### Question 02

Calculate the Euclidean distance between Brahms and Wagner in the `ex_hand` DataFrame. 


Assign the computed distance as a float to the variable ans2. Make sure that is answer is accurate to 3 decimal places. Feel free to use the KNN assignment as a reference.


In [ ]:
### GRADED
# Calculate the Euclidean distance between Brahms and Wagner in the `ex_hand` DataFrame
# Assign the computed distance as a float to the variable ans2. Make sure that is answer is 
# accurate to 3 decimal places. Feel free to use the KNN assignment as a reference.
### YOUR SOLUTION HERE

ans2 = ...
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


To normalize the Euclidean distance between 0 and 1, we can use the following formula that relates "distance" to "similarity"

$$\text{similarity} =\frac1{1+\text{distance}}.$$

For example, a distance of 0 corresponds to a similarity of 1, and a very large distance (e.g., $\infty$) corresponds to a similarity of 0.  

<a id="q03"></a>
[Return to top](#questions)
### Question 03

Define a  function called `e_sim` that accepts two `pandas` series as arguments.

Your function should return the euclidean similarity score between the two series. Make sure that is answer is accurate to 3 decimal places.

For this question you will need to use the attribute `np.linalg.norm()`.

In [ ]:
### GRADED
# Define a  function called `e_sim` that accepts two `pandas` series as arguments.
# Your function should return the euclidean similarity score between the two series. 
# Make sure that is answer is accurate to 3 decimal places.

# For this question you will need to use the attribute `np.linalg.norm()`.
### YOUR SOLUTION HERE

def e_sim(ser1, ser2):
    """
    Given two Pandas series, compute the euclidean similarity score;
        1 / 1+euclidean distance
        
    Positional Arguments --
        ser1: a Pandas Series of length n
        ser2: a Pandas Series of length n
    
    Example --
        ser1 = ex_hand.iloc[:,0]
        ser2 = ex_hand.iloc[:,1]
        print(e_sim(ser1, ser2)) #--> 0.28989794855663564
    """
###
### YOUR CODE HERE
###
    return np.nan

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


#### Pearson's Correlation Coefficient

The Pearson's correlation coefficient $\rho$ takes values from -1 to 1. Therefore, to normalize it into a difference score between 0 and 1, we need to reduce that range by half (i.e., divide the range by two) to increase the minimum score to 0, and add .5:  

$$\frac12 + \frac{\rho_{xy}}2$$  

The function below computes the Pearson's correlation coefficient.

In [ ]:
### Function to define the Pearson's Correlation Coefficient.
def p_sim(ser1, ser2):
    
    def normalize(raw):
        return .5 + (raw/2)
    
    corr = np.corrcoef(ser1, ser2)[0][1] ### returns 2x2 array with correlation to self(1) on diagonal
    
    return normalize(corr)

#### Cosine Similarity

The cosine similarity calculates the cosine of the angle between two vectors normalizing it between 0 and 1.  

<a id="q04"></a>
[Return to top](#questions)
### Question 04

Decide whether the following statement is true or false.

*Like the correlation coefficient similarity score, the cosine similarity score is unaffected by magnitudes. For example, an excited user who gives mostly 5's will be considered similar to a user who gives mostly 2's.*

Assign a Boolean value to the variable ans1.

In [ ]:
### GRADED
#Decide whether the following statement is true or false.

#*Like the correlation coefficient similarity score, the cosine similarity score is 
# unaffected by magnitudes. For example, an excited user who gives mostly 5's will be 
# considered similar to a user who gives mostly 2's.*

#Assign a Boolean value to the variable ans1.
### YOUR SOLUTION HERE

ans1 = ...
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


The cosine similarity score is given by the following formula

$$cos(\theta) = \frac{S1\cdot S2}{||S1||\cdot||S2||},$$
 
 where $S1$ and $S2$ are the two vectors considered, and $||\cdot||$ represents the magnitude of each vector.
 
The function below computes the cosine similarity score.

In [ ]:
###Function to define the cosine similarity.

def c_sim(ser1, ser2):
    def normalize(raw):
        return .5 + (raw/2)
    
    cosT = np.dot(ser1, ser2) / (np.linalg.norm(ser1)* np.linalg.norm(ser2))
    
    return normalize(cosT)

At this point we have defined three functions for calculating similarity between two vectors.  

- e_sim: for the Euclidean similarity 
- p_sim: for the Pearson correlation coefficient similarity
- c_sim: for the cosine similarity
 

Now, let's calculate each of the three similarity scores for each pair of musicians in the `ex` DataFrame.  

Let's remind us of what the DataFrame `ex` looks like.

In [ ]:
ex

The function we will build must take into account that not all users have rated all musicians.

So, for example, when comparing "Mozart" and "Bach", we must disregard the observations from "Abel" and "Baker", respectively.  

This can be done by reading the value "0" as to be "less than 1", and *not* as the lack of information which actually represents.  


<a id="q05"></a>
[Return to top](#questions)
### Question 05
Define a  function called `drop_val` that accepts the following  arguments:

- A `pandas` DataFrame.
- A numeric variable named `to_drop`.

Your function should drop all the rows that have the value contained in `to_drop` in any column, and should return the DataFrame without the rows that have been dropped.

For example, if we pass the DataFrame `ex` containing all rows but only the columns corresponding to "Mozart" and "Bach", and we set  'to_drop=5', our function should return "Abel", "Erik", and "Frank" rows with all the original columns. 

In [ ]:
### GRADED
#Define a function called drop_val that accepts the following arguments:
# - A pandas DataFrame.
# - A numeric variable named to_drop.

# Your function should drop all the rows that have the value contained in to_drop in any column, 
# and should return the DataFrame without the rows that have been dropped.

# For example, if we pass the DataFrame ex containing all rows but only the columns corresponding
# to "Mozart" and "Bach", and we set 'to_drop=5', our function should return "Abel", "Erik", and "Frank" 
# rows with all the original columns.
### YOUR SOLUTION HERE

def drop_val (df, to_drop):
    
    """
    Drop rows from the DataFrame containing the specified values
    
    Positional Arguments --
        df: a Pandas DataFrame
        to_drop: a value found in some rows of df
    
    Example --
        
        df = ex.loc[:,"Mozart":"Bach]
        to_drop = 5
        print(drop_val(df,to_drop)) # -->          Mozart  Bach
                                            Abel        0     1
                                            Erik        3     3
                                            Frank       2     2   
    """
###
### YOUR CODE HERE
###
    return pd.DataFrame()

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


The cell below defines a function "drop_rows_with_zeros" that takes a DataFrame with two columns and returns a DataFrame where all the rows that contain a value of 0 have been removed.

In [ ]:
### Defining drop_rows_with_zeros

def drop_rows_with_zeros(df):
    keep = np.intersect1d( np.nonzero(df.iloc[:,0]), np.nonzero(df.iloc[:,1]))
    
    return df.iloc[keep,:]


### Comparison of Similarity Scores

Below, we will compare the similarity scores for each each combination of musicians. 

First, we find all the possible pairs of musicians:

In [ ]:
import itertools
for c in itertools.combinations(ex.columns,2):
    print(c)

Each tuple can be used as a subset in the DataFrame. For example, the subset for Mozart and Bach is given by:

In [ ]:
ex[list(('Mozart', 'Bach'))]

Next, we define a first dictionary that has as keys the pairs musicians being compared.

In a second dictionary, we define the similarity scores, "Euclid", "Pearson", and "Cosine",  corresponding to each pair of musician.

In [ ]:
sim_scores = dict()

for c in itertools.combinations(ex.columns, 2):
    df = drop_rows_with_zeros(ex[list(c)])
    ser1 = df.iloc[:,0]
    ser2 = df.iloc[:,1]
    scores = {"Euclid":e_sim(ser1, ser2), "Pearson":p_sim(ser1,ser2), "Cosine":c_sim(ser1,ser2)}
    key =", ".join(c)
    sim_scores[key] = scores
    
sims = pd.DataFrame.from_dict(sim_scores,orient = "index")
sims

The similarity scores can be visualized as follow

In [ ]:
### Note: x and y scales are not consistent in this visualization
sns.pairplot(sims );

In [ ]:
### Visualization with consistent x and y scales using `matplotlib`
plt.figure(figsize = (12,6))
for i, cols in enumerate([('Euclid','Pearson'), ("Euclid","Cosine"),("Pearson","Cosine")]):
    plt.subplot(1,3,i+1)
    
    plt.scatter(sims[cols[0]], sims[cols[1]])
    
    plt.xlabel(cols[0], fontsize = 14); plt.ylabel(cols[1], fontsize = 14)
    plt.xlim(0,1); plt.ylim(0,1)
    
plt.tight_layout()

<a id = "sim-note"></a>
#### Differences in Similarity Scores


As we can see from the graphs above, it is possible to visualize a range of similarity scores given different techniques. These similarity scores will majorly impact how a recommender system performs. 

Refer to the lectures for the particular impacts and strengths of the different similarity calculations.

<a id = 'pred'></a>
### Predicting Unknown Scores

Assume that we don't know the score that Abel gave to Mozart. To do so, we must take into account that Mozart has a similarity score in relation to all the other musicians.

Similarly, Abel has given a score to all the other musicians.  

Therefore, Abel's predicted score for Mozart will be given by: the **sum** of Abel's score for every other musician **times** that musician's similarity score with respect to Mozart, **divided by** the sum of similarity scores.  

For example, we can predict Abel's score for Mozart by computing

$$\frac{\sum_{ mus = Bach}^{Liszt}A_{mus}*\text{simScore}(\text{Mozart, mus})}{\sum_{mus = Bach}^{Liszt}\text{simScore}{\text{(Mozart, mus)}}}$$ 

The commands below compute the similarity scores for Mozart.

In [ ]:
mozSimScores = {}
for mus in ex.columns[1:]:
    no_zeros = drop_rows_with_zeros(ex[['Mozart', mus]])
    
    mozSimScores[mus] = round(p_sim(no_zeros.iloc[:,0], no_zeros.iloc[:,1]),2)

print(mozSimScores)

<a id="q06"></a>
[Return to top](#questions)
### Question 06

Given the similarity scores calculated and stored in `mozSimScores`, use the above formula to predict Abel's score for Mozart. Assign your result to the variable ans1.Make sure that is answer is accurate to 3 decimal places.

*Hint*: Use the rounded score computed above.

In [ ]:
### GRADED
# Given the similarity scores calculated and stored in `mozSimScores`, use the above 
# formula to predict Abel's score for Mozart. Assign your result to the variable ans1.
# Make sure that is answer is accurate to 3 decimal places.


# *Hint*: Use the rounded score computed above.

###  YOUR SOLUTION HERE

ans1 = ...
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


We observe that, although many other users gave Mozart high scores, because Abel appears to be less generous in his reviews, his predicted score for Mozart is fairly low. 

The function defined below predicts an unknown score based on the formula above.

In [ ]:
### Creating a function to create predictions in the manner described above.
### Note: "user" is a row, and "item" is a column in the df.
### simFunc defaults to p_sim

def scorePred(df, user, item, simFunc = p_sim, rev_items = None):
    """
    Positional Arguments --
        df: Pandas DataFrame
        user: Row-index in df
        item: Column-index in df
        simFunc: a similarity function (e_sim, p_sim, or c_sim)
        rev_items: For larger dfs, specifies all items reviewed by "user"
    """
    # Check to see if user has already scored item
    if df.loc[user,item] > 0:
        return "Already rated a "+str(df.loc[user,item])
    
    # rev_items used for larger DataFrame,
    # when you have all the items a particular user has reviewed already
    # otherwise, if "None" (if statement below)
    # take all other items other than item to predict
    if not rev_items:
        rev_items = set(df.columns)
        rev_items.remove(item)
    
    sim_total, user_sim_total = 0,0
    
    for other_item in rev_items:
        user_score_other_item = df.loc[user, other_item] # grab user score
        
        if user_score_other_item == 0:
            print("no user score")
            continue
        
        # Use function built above to drop all other users with "other_item" score of "0"
        no_zeros = drop_rows_with_zeros(df[[item, other_item]]) 
        sh = no_zeros.shape
        
        # If no other users, move to next item
        if sh[0] == 0:
            continue
            
        #print(no_zeros.shape)
        
        # calculate similarity score using non-zero information. 
        ser1 = no_zeros.iloc[:,0]
        ser2 = no_zeros.iloc[:,1]
        # print(ser1, ser2)
        ss = simFunc(no_zeros.iloc[:,0], no_zeros.iloc[:,1])
        # print(ss, user_score_other_item)
        
        # add up sim total and weighted sim total
        sim_total += ss
        user_sim_total += user_score_other_item * ss

        
    if sim_total == 0:
        return 0
    
    else:
        return user_sim_total / sim_total

In [ ]:
ex

Use the cells below to compute the unknown score for different cases using the function `scorePred` defined above.

In [ ]:
scorePred(ex, 'Baker', 'Bach')

<a id = "ret"></a>
### Returning to the Data
Let's go back to the Amazon rating DataFrame

In [ ]:
rev_df.head(10)

We observe that now the data does not conform to the two-dimensional n-by-p matrix that we have been using. Therefore, we need to manipulate our data. To do so we will use `Pandas` built-in `pivot()` attribute.

In [ ]:
# Reviewers on rows, item on cols. 
reshaped_reviews = rev_df.pivot(index = 'reviewerID', columns = 'productID')
reshaped_reviews.fillna(0, inplace = True) # turns NA into 0
reshaped_reviews.columns = reshaped_reviews.columns.droplevel() # removes artifact of pivot function


print("shape: ", reshaped_reviews.shape)
reshaped_reviews.head()

It should be clear from the output above that the majority of the entries corresponding to a certain pair of costumer and product are now empty. Thus, when trying to provide recommendations, it will not be possible to predict scores for every  combination of costumer and product. In fact, to predict scores, we require a certain amount of overlap between the data.

Below predictions are created for our reviewer called "R0"

In [ ]:
### First, find the products that 'R0' has reviewed.
### These are the products that will be used for creating similarity scores.

r0 = reshaped_reviews.loc["R0", :]
# numpy function returns all indexes that are not "0"
r0_nonzero = np.nonzero(r0)
# collect names of reviewed products
r0_rev_prods = list(r0.iloc[r0_nonzero].index) 
print(len(r0_rev_prods))
r0_rev_prods[:5]

Next, we create a list containing the product that have not been reviewed by R0.

In [ ]:
### Creating a list of products not reviewed by R0

# return list of all prods not in the "reviewed" list
r0_not_rev = np.setdiff1d(reshaped_reviews.columns, r0_rev_prods) 

print(len(r0_not_rev))
r0_not_rev

The commands below requires too much processing for Vocareum. We simply display the output below:

In [ ]:
# %%time
# ### predict scores for products not yet reviewed by R0, using function built above: `scorePred`
# ### cosine similarity used because if a product all has only one value of review (e.g. all 5s)
# ### then the standard_deviation is 0 and the correlation coefficient is not calculable

# preds = {}
# for item in r0_not_rev:
#     preds[item] = scorePred(reshaped_reviews, 'R0', item, simFunc = c_sim, rev_items = r0_rev_prods)

In [ ]:
# pd.Series(preds).sort_values(ascending = False).head(10)

Running the above cells took the local machine ~7.5'. results in the picture below.
![preds](./data/r0_preds.PNG)

<a id = "util"></a>
### Utilization Complexities

Certainly, if  a product is likely to score a "5", then that product should be recommended. However, because we are dealing with very sparse data, the confidence in our predictions will vary between each product.

<a id="q07"></a>
[Return to top](#questions)
### Question 07

Consider 3 products: `A`, `B`, and `X`. Assume that `userN` has ranked both products `A`, and `B` with a score of 5. Assume that `user1` has ranked products `A`, and `X` both with score 3 and assume that user2 has ranked products `B` and `X` with score 2.

What will be the similarity score between product `A` and product `X`? Assign the result the the variable ans1.

What will be the similarity score between product `B` and product `X`? Assign the result the the variable ans2.


In [ ]:
### GRADED
# Consider 3 products: `A`, `B`, and `X`. Assume that `userN` has ranked both products `A`, and `B` 
# with a score of 5. Assume that `user1` has ranked products `A`, and `X` both with score 3 and assume 
# that user2 has ranked products `B` and `X` with score 2.

# What will be the similarity score between product `A` and product `X`? Assign the result the the variable ans1.

#What will be the similarity score between product `B` and product `X`? Assign the result the the variable ans2.
### YOUR SOLUTION HERE

ans1 = ...
ans2 = ...
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


<a id="q08"></a>
[Return to top](#questions)
### Question 08

Given your answer above, what will be the predicted score of `userN` for product `X`?

Assign your result to the variable ans1.

In [ ]:
### GRADED
#Given your answer above, what will be the predicted score of `userN` for product `X`?

#Assign your result to the variable ans1.
###  YOUR SOLUTION HERE
ans1 = ...
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


it is important to note that we could have run into other possible scenarios: high similarity scores because of few overlapping reviews, or high-user score for the user who is being predicted for.  

<a id = "svd"></a>
### Singular Value Decomposition (SVD)

We can deal with the sparsity of data and, at the same time, increase the speed of our algorithm,  by using Singular Value Decomposition (SVD). SVD is a technique that reduces the dimensionality of functions by "capturing" a majority of the information present in a dataset in a smaller number of variables.  

SVD relies upon some straight-forward linear algebra that is worth understanding. The main concept have been covered  in the lectures. Here are some more insightful resources about SVD.  

[Here is a nice introduction.](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/)  

[Below, we will use `numpy's` implementation of SVD](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html)

We have learned that the quantity $\Sigma ^2$ describes the variance of each vector. The plot below displays the rate of the variance given by n vectors as a function of the total number of products.

![svd](./assets/svd.PNG)

We notice that about 1700 of the decomposed vectors will give around 80% of the variance in the data. Computationally, this will make the calculations much faster: in fact, considering only 1/3 of the vectors will yield 80% of the value. 

In [ ]:
# # Perform SVD
# u, sigma, _ = np.linalg.svd(reshaped_reviews)

# # Create function to decompose DataFrame
# def return_svd(df , u, sigma, n):
    
#     sigN = np.mat(np.eye(n) * sigma[:n]) #arrange Sig4 into a diagonal matrix
    
#     n_svd_vectors = np.dot(df.T, np.dot(u[:,:n] , sigN.I))  #create transformed items
    
#     return pd.DataFrame(n_svd_vectors).T

# # use function to create decomposed DataFrame
# svd_df = return_svd(reshaped_reviews, u, sigma, 1700)
# svd_df.columns = reshaped_reviews.columns

# # Save DataFrame

# svd_df.to_csv("svd_df.csv")

The commands above take too long to run for Vocareum. The results are displayed below.

In [ ]:
svd_df = pd.read_csv("./data/svd_df.csv", index_col = 0)
svd_df.head()

As `svd_df` indicates, the svd vectors correspond to different items. The calculation for the recommendation will be the same, but will be done by using these new values.  

For example, to predict the score that reviewer "R0" would give to  the product 'P1' (Remember "R0" already reviewed "P0") we must follow the following steps:  

1. Find the products that R0 scored.
2. Find the similarity between those scored products and "P1"
3. Multiply those similarity scores by the scores from R0
4. Divide by sum of similarity scores.  

The downside of SVD is that it must be performed before the calculations, and, as was seen above, such computations can take some time. In fact, in practice, SVD would need to be performed each time a new review is added.

The following function computes the prediction scores from the `svd_df` DataFrame.

In [ ]:
### Creating a function to create predictions using the SVD_df
### Note: "user" is a row, and "item" is a column in the df, and a column in svd_df.
### simFunc defaults to p_sim

def scorePredSVD(df, user, item, svd_df, simFunc = p_sim, rev_items = None):
    
    # Check to see if user has already scored item
    if df.loc[user,item] > 0:
        return "Already rated a "+str(df.loc[user,item])
    
    # Code below should be familiar from "scorePred" defined above
    if not rev_items:
        rev_items = set(df.columns)
        rev_items.remove(item)
    
    sim_total, user_sim_total = 0,0
    
    for other_item in rev_items:
        user_score_other_item = df.loc[user, other_item]
        
        if user_score_other_item == 0:
            print("no user score")
            continue
            
        ser1 = svd_df.loc[:,item]
        ser2 = svd_df.loc[:,other_item]
        # print(ser1, ser2)
        ss = simFunc(ser1, ser2)
        # print(ss, user_score_other_item)
        sim_total += ss
        user_sim_total += user_score_other_item * ss

        
    if sim_total == 0:
        return 0
    
    else:
        return user_sim_total / sim_total

The commands below predict  the scores for products not yet reviewed by R0, using function built above: scorePred.
We use the cosine similarity because if a product has only one value of review (e.g. all 5s), then the standard deviation is 0 and the correlation coefficient is not calculable.

In [ ]:
%%time

SVDpreds = {}
for item in r0_not_rev:
    SVDpreds[item] = scorePredSVD(reshaped_reviews, 'R0', item, svd_df, simFunc = c_sim, rev_items = r0_rev_prods)

In [ ]:
pd.Series(SVDpreds).sort_values(ascending = False).head()

<a id="q09"></a>
[Return to top](#questions)
### Question 09

SVD is a kind of:
- a) Variance reduction.
- b) Special type of array in Python.
- c) Dimensionality reduction.
- d) Description of central tendency.

Assign the character associated with your choice as a string to the variable ans1.

In [ ]:
### GRADED
# SVD is a kind of:
# a) Variance reduction.
# b) Special type of array in Python.
# c) Dimensionality reduction.
# d) Description of central tendency.

#Assign the character associated with your choice as a string to the variable ans1.
### YOUR SOLUTION HERE

ans1 = ''
###
### YOUR CODE HERE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


<a id = "surprise"></a>
### Surprise - Python Package
[`Surprise`](http://surpriselib.com) is a Python package which implements a number of tools for building and testing recommender systems. The documentation can be found [here](https://surprise.readthedocs.io/en/stable/).

**THE FOLLOWING CODE WILL NOT RUN ON VOCAREUM. WE JUST PROVIDE EXAMPLES IN CASE YOU CHOOSE TO USE `SURPRISE` IN YOUR OWN ENVIRONMENT**  


To install surprise, run:  

`conda install -c conda-forge scikit-surprise`  

in the Anaconda Prompt. After the installation, you might have to reinstall or update scipy, and restart Kernel by doing

`conda install -c anaconda scipy`  

Below is [example code from the surprise documentation](https://surprise.readthedocs.io/en/stable/getting_started.html#)

In [ ]:
# from surprise import SVD
# from surprise import Dataset
# from surprise.model_selection import cross_validate


# # Load the movielens-100k dataset (download it if needed),
# data = Dataset.load_builtin('ml-100k')

# # We'll use the famous SVD algorithm.
# algo = SVD()

# # Run 5-fold cross-validation and print results
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

[Loading in a df](https://surprise.readthedocs.io/en/stable/dataset.html?highlight=pandas#surprise.dataset.Dataset.load_from_df)

In [ ]:
# import os
# from surprise import BaselineOnly
# from surprise import Reader

# # when using a df to load in data, the columns must be:
# # user, product, rating, in that order
# # a reader must be defined to describe the rating_scale
# reader = Reader(rating_scale=(0,5))

# data = Dataset.load_from_df(rev_df, reader=reader)

# # We can now use this dataset as we please, e.g. calling cross_validate
# cross_validate(BaselineOnly(), data, verbose=True)

### One final note: 

This project has based its recommendations of comparing the similarity of different *items*.  Using the same technique, it is also possible to calculate the similarity of *users*. The reason why *items* is usually the default choice is that there are usually fewer items than users. This decreases the number of similarity calculations required. 

For example, in our data, we had roughly 3.5k items and 5.5k users. When using a complete dataset from a specific source, this difference will be more significant.